In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix, r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import os
import sys
sys.path.append(os.path.abspath('../'))
from modules import mlp

# Датасет Titanic

## Подготовка данных

Импортируем датасет

In [3]:
data = pd.read_csv('../Data/titanic.csv', sep = ',')

In [4]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
data.fillna(data.mean(), inplace=True)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [6]:
data = pd.get_dummies(data=data, prefix='c', columns=['Sex', 'Embarked'])
data

,Survived,Pclass,Age,SibSp,Parch,Fare,c_female,c_male,c_C,c_Q,c_S
0,0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,1,35.000000,1,0,53.1000,1,0,0,0,1
4,0,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,0,1,0,0,1
887,1,1,19.000000,0,0,30.0000,1,0,0,0,1
888,0,3,29.699118,1,2,23.4500,1,0,0,0,1
889,1,1,26.000000,0,0,30.0000,0,1,1,0,0


In [7]:
y = data[['Survived']]
X = data[data.columns[1:data.shape[1]]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

## Классификация с помощью модели MLP

In [8]:
model = mlp.MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(15, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative),
                                                                      (15, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train.values, y_train.values, epochs=600, learning_rate=0.25)

In [9]:
y_pred = []
for i in range(len(X_test.values)):
    prediction = model.predict(X_test.values[i])[0]
    if prediction > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [10]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[110   0]
 [ 69   0]]
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       110
           1       0.00      0.00      0.00        69

    accuracy                           0.61       179
   macro avg       0.31      0.50      0.38       179
weighted avg       0.38      0.61      0.47       179

C:\Users\vipki\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vipki\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

# Датасет Iris

## Подготовка данных

 Импортируем датасет

In [15]:
iris = datasets.load_iris()

In [16]:
X = iris.data
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [17]:
y = iris.target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [18]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##  Классификация с помощью модели MLP

In [20]:
model = mlp.MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative),
                                                                      (3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)],
                outputs_count_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train, y_train, epochs=300, learning_rate=0.15)

In [21]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(np.argmax(model.predict(X_test[i])))

In [22]:
 print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10  0  0]
 [ 4  6  0]
 [10  0  0]]
              precision    recall  f1-score   support

           0       0.42      1.00      0.59        10
           1       1.00      0.60      0.75        10
           2       0.00      0.00      0.00        10

    accuracy                           0.53        30
   macro avg       0.47      0.53      0.45        30
weighted avg       0.47      0.53      0.45        30

C:\Users\vipki\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vipki\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `

# Bank Marketing Data Set

## Подготовка данных

 Импортируем датасет

In [24]:
bank_df = pd.read_csv('../Data/Bank Marketing Data Set/bank_preprocessed.csv', sep = ',')

In [25]:
bank_df

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y_no,y_yes
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
1,33,4789,11,220,1,339,4,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,30,1476,3,199,4,-1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,59,0,5,226,1,-1,0,0,1,0,...,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,-333,30,329,5,-1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4517,57,-3313,9,153,1,-1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
4518,57,295,19,151,11,-1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4519,28,1137,6,129,4,211,3,0,1,0,...,0,0,0,0,0,1,0,0,1,0


In [26]:
X = bank_df.iloc[:, :-1].values
print(X)

[[  30 1787   19 ...    0    1    1]
 [  33 4789   11 ...    0    0    1]
 [  35 1350   16 ...    0    0    1]
 ...
 [  57  295   19 ...    0    1    1]
 [  28 1137    6 ...    0    0    1]
 [  44 1136    3 ...    0    0    1]]


In [27]:
y = bank_df.iloc[:, 8].values
print(y)

[0 0 0 ... 0 1 0]


Разбиваем исходные данные на обучающую и контрольную выборки. Обучающая выборка составляет 80% от объема исходных данных, а контрольная составляет, соответственно, 20%

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Объявляем StandardScaler для нормализации данных

In [29]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Модель MLP

In [30]:
model = mlp.MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative),
                                                                      (3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)],
                outputs_count_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train, y_train, epochs=300, learning_rate=0.15)

In [31]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(np.argmax(model.predict(X_test[i])))

In [32]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 20   0 691]
 [  1 193   0]
 [  0   0   0]]
              precision    recall  f1-score   support

           0       0.95      0.03      0.05       711
           1       1.00      0.99      1.00       194
           2       0.00      0.00      0.00         0

    accuracy                           0.24       905
   macro avg       0.65      0.34      0.35       905
weighted avg       0.96      0.24      0.26       905

C:\Users\vipki\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vipki\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_di

## Выводы
 
Выполнена своя реализация многослойного персептрона. 
Модель протестирована на рекомендуемых датасетах. 
Качество классификации получилось заметно хуже, чем при использовании базовых моделей